In [20]:
import pandas as pd
from sklearn.utils import resample
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
import numpy as np
from sklearn.impute import SimpleImputer
import joblib
from xgboost import XGBClassifier
import optuna
from optuna.samplers import TPESampler


In [21]:
#load dataset
df = pd.read_csv("../data/processed/home_credit_train_ready.csv")

In [22]:
#rebalance dataset
class_0 = df[df['TARGET'] == 0]
class_1 = df[df['TARGET'] == 1]

# balance mayority class with minority
sub_class = resample(class_0, replace=False, n_samples=len(class_1), random_state=42)

# combine
df = pd.concat([sub_class, class_1])

In [23]:
y = df['TARGET']
X = df.drop(columns=['TARGET', 'SK_ID_CURR'])  #adde sk_id_curr

In [24]:
#manages divided by 0 values
X.replace([np.inf, -np.inf], np.nan, inplace=True)

In [25]:
#divide categorical and numerical to impute the values
cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

num_imputer = SimpleImputer(strategy="median")  #change median for mean
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")

X[num_cols] = num_imputer.fit_transform(X[num_cols])
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

In [26]:
#change object to category to use in the threelike model
for col in X.select_dtypes("object"):
    X[col] = X[col].astype("category")

In [27]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y ,random_state=42)

In [28]:
#create params for optuna study
from catboost import CatBoostClassifier

# Identificar columnas categóricas
cat_features = X_train.select_dtypes(include=['category', 'object']).columns.tolist()

catboost_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'PRAUC',
    'custom_metric': ['AUC', 'PRAUC'],

    'auto_class_weights': 'Balanced',

    # Boosting
    'iterations': 4000,
    'learning_rate': 0.025,
    'depth': 7,

    # Regularización (compensa falta de rsm)
    'l2_leaf_reg': 5.0,
    'random_strength': 1.5,

    # Bootstrap
    'bootstrap_type': 'Bayesian',
    'bagging_temperature': 0.7,

    # Growing
    'grow_policy': 'SymmetricTree',
    'min_data_in_leaf': 15,

    # Categóricas
    'cat_features': cat_features,
    'one_hot_max_size': 12,

    # GPU
    'task_type': 'GPU',
    'devices': '0',

    # Control
    'random_seed': 42,
    'early_stopping_rounds': 250,
    'verbose': 100,
    'use_best_model': True,}


In [29]:

model = CatBoostClassifier(**catboost_params)
model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    verbose=200
)

Default metric period is 5 because AUC, PRAUC is/are not implemented for GPU
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PRAUC is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.7219027	test: 0.7100887	best: 0.7100887 (0)	total: 90.8ms	remaining: 6m 3s
200:	learn: 0.7836377	test: 0.7540492	best: 0.7540492 (200)	total: 17.5s	remaining: 5m 30s
400:	learn: 0.8023959	test: 0.7606448	best: 0.7607023 (389)	total: 34.1s	remaining: 5m 5s
600:	learn: 0.8143390	test: 0.7619499	best: 0.7619838 (595)	total: 49.8s	remaining: 4m 41s
800:	learn: 0.8264981	test: 0.7640267	best: 0.7640267 (800)	total: 1m 6s	remaining: 4m 23s
1000:	learn: 0.8367757	test: 0.7649480	best: 0.7650519 (988)	total: 1m 22s	remaining: 4m 5s
1200:	learn: 0.8468329	test: 0.7659624	best: 0.7660221 (1197)	total: 1m 38s	remaining: 3m 48s
1400:	learn: 0.8539803	test: 0.7659685	best: 0.7660769 (1337)	total: 1m 53s	remaining: 3m 31s
1600:	learn: 0.8607275	test: 0.7662084	best: 0.7662084 (1595)	total: 2m 9s	remaining: 3m 14s
1800:	learn: 0.8679429	test: 0.7666165	best: 0.7666165 (1800)	total: 2m 25s	remaining: 2m 57s
2000:	learn: 0.8757182	test: 0.7670526	best: 0.7671124 (1983)	total: 2m 41s	remaini

In [30]:

# Probabilidades (idéntico conceptualmente)
y_val_proba = model.predict_proba(X_val)[:, 1]

# AUC (esto es lo importante)
auc = roc_auc_score(y_val, y_val_proba)

#optimal threshold
thresholds = np.linspace(0.05, 0.6, 100)
f1_scores = [f1_score(y_val, y_val_proba >= t) for t in thresholds]


optimal_threshold = thresholds[np.argmax(f1_scores)]
print(f'optimal threshold: {optimal_threshold}')

#Threshold (solo para métricas de clasificación)
adj_threshold = optimal_threshold

y_val_pred = (y_val_proba >= adj_threshold).astype(int)

# Classification report
report = classification_report(y_val, y_val_pred)

#Confusion matrix manual
TP = np.sum((y_val == 1) & (y_val_pred == 1))
FP = np.sum((y_val == 0) & (y_val_pred == 1))
TN = np.sum((y_val == 0) & (y_val_pred == 0))
FN = np.sum((y_val == 1) & (y_val_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(report)

cm = confusion_matrix(y_val, y_val_pred)
print(f"\n📊 Matriz de Confusión:")
print(f"                 Predicho")
print(f"               No Def  Default")
print(f"Real No Def    {cm[0, 0]:6d}   {cm[0, 1]:6d}")
print(f"Real Default   {cm[1, 0]:6d}   {cm[1, 1]:6d}")

optimal threshold: 0.41111111111111104
FPR: 0.4167
Precision (PPV): 0.6637
AUC validación: 0.77926
              precision    recall  f1-score   support

           0       0.77      0.58      0.66      2184
           1       0.66      0.82      0.73      2183

    accuracy                           0.70      4367
   macro avg       0.72      0.70      0.70      4367
weighted avg       0.72      0.70      0.70      4367


📊 Matriz de Confusión:
                 Predicho
               No Def  Default
Real No Def      1274      910
Real Default      387     1796


In [39]:
df = pd.read_csv("../data/processed/home_credit_train_ready.csv")

y = df['TARGET']
X = df.drop(columns = ['TARGET', 'SK_ID_CURR'])

X.replace([np.inf, -np.inf], np.nan, inplace=True)

cat_cols = X.select_dtypes(include=["object"]).columns
num_cols = X.select_dtypes(exclude=["object"]).columns

num_imputer = SimpleImputer(strategy="median")  #change median for mean
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")

X[num_cols] = num_imputer.fit_transform(X[num_cols])
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

for col in X.select_dtypes("object"):
    X[col] = X[col].astype("category")

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.6, stratify=y ,random_state=42)

# Probabilidades (idéntico conceptualmente)
y_val_proba = model.predict_proba(X_val)[:, 1]

# AUC (esto es lo importante)
auc = roc_auc_score(y_val, y_val_proba)

#optimal threshold based on expected loss
cost_FN = 10000   # default
cost_FP = 1000    # lost good client

losses = []

for t in thresholds:
    y_pred = (y_val_proba >= t).astype(int)

    FP = np.sum((y_val == 0) & (y_pred == 1))
    FN = np.sum((y_val == 1) & (y_pred == 0))

    loss = FP * cost_FP + FN * cost_FN
    losses.append(loss)

optimal_threshold = thresholds[np.argmin(losses)]



#Threshold (solo para métricas de clasificación)
adj_threshold = optimal_threshold

y_val_pred = (y_val_proba >= adj_threshold).astype(int)

# Classification report
report = classification_report(y_val, y_val_pred)

#Confusion matrix manual
TP = np.sum((y_val == 1) & (y_val_pred == 1))
FP = np.sum((y_val == 0) & (y_val_pred == 1))
TN = np.sum((y_val == 0) & (y_val_pred == 0))
FN = np.sum((y_val == 1) & (y_val_pred == 0))

FPR = FP / (FP + TN)
Precision = TP / (TP + FP)

print(f'used threshold: {adj_threshold}')
print(f"FPR: {FPR:.4f}")
print(f"Precision (PPV): {Precision:.4f}")
print(f"AUC validación: {auc:.5f}")
print(report)

cm = confusion_matrix(y_val, y_val_pred)
print(f"\n📊 Matriz de Confusión:")
print(f"                 Predicho")
print(f"               No Def  Default")
print(f"Real No Def    {cm[0, 0]:6d}   {cm[0, 1]:6d}")
print(f"Real Default   {cm[1, 0]:6d}   {cm[1, 1]:6d}")


used threshold: 0.47929292929292927
FPR: 0.2896
Precision (PPV): 0.2032
AUC validación: 0.82251
              precision    recall  f1-score   support

           0       0.97      0.71      0.82    138182
           1       0.20      0.78      0.32     13101

    accuracy                           0.72    151283
   macro avg       0.59      0.74      0.57    151283
weighted avg       0.90      0.72      0.78    151283


📊 Matriz de Confusión:
                 Predicho
               No Def  Default
Real No Def     98171    40011
Real Default     2899    10202


In [40]:
joblib.dump(model, '../models/catboost_new_dataset.pkl')

['../models/catboost_new_dataset.pkl']